# Regrese s využitím neuronový sítí


Pro ilustraci použití neuronových sítí v knihovně [TensorFlow](https://www.tensorflow.org/) použijeme opět data set cen nemovitostí.

## Načteme trénovací a testovací data do DataFramu Pandas.

* **Stav** je stav budovy na škále 1 až 10, kde 1 je nejhorší a 10 nejlepší.
* **plocha** je obytná plocha budovy v metrech čtverečních
* **rok_stavby** je rok výstavby domu
* **rok_prodeje** je rok, ve kterém byl dům prodaný
* **cena** je cena, za kterou se dům prodal 

In [1]:
import pandas as pd
data = pd.read_csv("https://raw.githubusercontent.com/mlcollege/ai-academy/main/06-Regrese/data/ceny_domu.csv", sep=',')
data.head()

,kvalita,plocha,datum_stavby,datum_prodeje,cena
0,7,158,2003,2008,5.21
1,6,117,1976,2007,4.54
2,7,165,2001,2008,5.59
3,7,159,1915,2006,3.50
4,8,204,2000,2008,6.25


## Rozdělíme si data na trénovací a testovací

In [2]:
from sklearn.model_selection import train_test_split

input_features = ['kvalita', 'plocha', 'datum_stavby',	'datum_prodeje']
target_feature = 'cena'

X_train, X_test, y_train, y_test = train_test_split(data[input_features], data[target_feature], test_size=0.1)
print('Velikost trenovaci mnoziny: {}'.format(len(X_train)))
print('Velikost testovaci mnoziny: {}'.format(len(X_test)))

Velikost trenovaci mnoziny: 1314
Velikost testovaci mnoziny: 146


## Vytvoříme a natrénujeme model neuronové sítě



Nejprve je vhodné data standardizovat stejným zůsobem jako jsme to dělali u logistické regrese. K tomu můžeme použití ScikitLearn.

In [3]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

Nyní můžeme vytvořit model neuronové sítě. Pouužijeme model se třemi vrstvami, přičemž první vrstva bude mít 30 neuronů a aktivační funkci _tanh_, druhá vrstva rovněž a poslední vrstva bude mít pouze jediný neuron s lineární aktivační funkcí. 

In [4]:
%tensorflow_version 2.x
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Activation

model = Sequential()

model.add(Dense(30, input_shape=(4, )))
model.add(Activation('tanh'))
model.add(Dense(30))
model.add(Activation('tanh'))
model.add(Dense(1))
model.add(Activation('linear'))

Dále je potřeba model zkompilovat. Pro kompilaci musíme zadat několik důležitých parametrů:
* **loss** - chybová funknce, kterou budeme minimalizovat. Vzhledem k tomu, že se jedná o regresi, bude vhodnou funkcí Mean Squared Error.
* **optimizer** - algoritmů pro hledání minima chybové funkce existuje celá řada. Všechny vycházejí z podobných principů a liší se hlavně ve způsobech, jakými minimalizují šanci uvíznutí v nějakém špatném lokálním extrému. Jedním z nejčastěji používaných optimalizátorů je [Adam](https://keras.io/api/optimizers/adam/).
* **metrics** - během trénování umožňuje algoritmus průběžné vypisování aktuální kvality modelu. Pokud o to stojíme, je třeba specifikovat konkrétní metriku. V našem případě jsme zvolili Mean Absolute Error.

In [5]:
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['mae'])

Samotné trénování se provádí metodou fit a je třeba opět specifikovat několik parametrů:

* trénovací data - X_train_std a y_train
* **batch_size** - počet trénovacích příkladů, které jsou zpracovávány současně.
* **epochs** - počet epoch (průchodů celými trénovacími daty).
* **verbose** - úroveň množství výpisů během trénování
* **validation_data** - data, použitá pro validaci

In [6]:
model.fit(X_train_std, y_train,
          batch_size = 64, epochs = 100, verbose=1,
          validation_data=(X_test_std, y_test))

Epoch 1/100
21/21 [==============================] - 1s 11ms/step - loss: 21.7230 - mae: 4.3664 - val_loss: 22.4440 - val_mae: 4.4201
Epoch 2/100
21/21 [==============================] - 0s 2ms/step - loss: 17.6982 - mae: 4.0447 - val_loss: 18.6094 - val_mae: 4.0331
Epoch 3/100
21/21 [==============================] - 0s 3ms/step - loss: 14.0564 - mae: 3.5819 - val_loss: 14.2687 - val_mae: 3.4544
Epoch 4/100
21/21 [==============================] - 0s 3ms/step - loss: 9.6357 - mae: 2.8917 - val_loss: 9.2776 - val_mae: 2.6184
Epoch 5/100
21/21 [==============================] - 0s 3ms/step - loss: 5.0929 - mae: 1.9614 - val_loss: 4.8908 - val_mae: 1.6077
Epoch 6/100
21/21 [==============================] - 0s 2ms/step - loss: 2.2017 - mae: 1.0751 - val_loss: 2.8643 - val_mae: 0.9580
Epoch 7/100
21/21 [==============================] - 0s 3ms/step - loss: 1.3356 - mae: 0.7751 - val_loss: 2.4075 - val_mae: 0.8875
Epoch 8/100
21/21 [==============================] - 0s 3ms/step - loss: 1.2

## Vyhodnocení kvality modelu

Nakonec provedeme predikce pro testovací data a vyhodnotíme výsledky. Pro vyhodnocení použijeme MAE i RMSE. Výsledky můžeme porovnat i s kvalitou dosaženou pomocí linární regrese a rozhodovacích stromů.

In [7]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = model.predict(X_test_std)

print ("Vyhodocnoceni kvality MAE: {}, MSE: {}".format(mean_absolute_error(y_test, y_pred), mean_squared_error(y_test, y_pred, squared=False)))

Vyhodocnoceni kvality MAE: 3.1081782491301015, MSE: 3.6889109957126633


Výsledky z linární regrsese a rozhodovacích stromů jsou následující:

Linearni regrese MAE: 0.6578472596904307, MSE: 0.8588766801030574

Random forest MAE: 0.5328928962818005, MSE: 0.74871399470129

Kvalita neuronových sítí pro tuto úlohu je tedy výrazně lepší než u lineární regese a srovnatelná s rozhodovacími stromy.